##### Manual TOOL creation script

In [ ]:
from typing import Callable


class Tool:
    """
    A class representing a reusable piece of code (Tool).

    Attributes:
        name (str): Name of the tool.
        description (str): A textual description of what the tool does.
        func (callable): The function this tool wraps.
        arguments (list): A list of argument.
        outputs (str or list): The return type(s) of the wrapped function.
    """

    def __init__(
        self, name: str, description: str, func: Callable, arguments: list, outputs: str
    ):
        self.name = name
        self.description = description
        self.func = func
        self.arguments = arguments
        self.outputs = outputs

    def to_string(self) -> str:
        """
        Return a string representation of the tool,
        including its name, description, arguments, and outputs.
        """
        args_str = ", ".join(
            [f"{arg_name}: {arg_type}" for arg_name, arg_type in self.arguments]
        )

        return (
            f"Tool Name: {self.name},"
            f" Description: {self.description},"
            f" Arguments: {args_str},"
            f" Outputs: {self.outputs}"
        )

    def __call__(self, *args, **kwargs):
        """
        Invoke the underlying function (callable) with provided arguments.
        """
        return self.func(*args, **kwargs)

In [ ]:
def calculator(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b


# Example usage
calculator_tool = Tool(
    name="Calculator",
    description="A simple calculator that multiplies two integers.",
    func=calculator,
    arguments=[("a", "int"), ("b", "int")],
    outputs="int",
)
print(calculator_tool.to_string())

In [ ]:
# decorator to create a Tool instance from a function

import inspect


def tool(func):
    """
    A decorator that creates a Tool instance from the given function.
    """
    # Get the function signature
    signature = inspect.signature(func)

    # Extract (param_name, param_annotation) pairs for inputs
    arguments = []
    for param in signature.parameters.values():
        annotation_name = (
            param.annotation.__name__
            if hasattr(param.annotation, "__name__")
            else str(param.annotation)
        )
        arguments.append((param.name, annotation_name))

    # Determine the return annotation
    return_annotation = signature.return_annotation
    if return_annotation is inspect._empty:
        outputs = "No return annotation"
    else:
        outputs = (
            return_annotation.__name__
            if hasattr(return_annotation, "__name__")
            else str(return_annotation)
        )

    # Use the function's docstring as the description (default if None)
    description = func.__doc__ or "No description provided."

    # The function name becomes the Tool name
    name = func.__name__

    # Return a new Tool instance
    return Tool(
        name=name,
        description=description,
        func=func,
        arguments=arguments,
        outputs=outputs,
    )

In [ ]:
@tool
def calculator(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b


print(calculator.to_string())

#### **Tool Calling With HuggingFaceTB/SmolLM2-1.7B-Instruct**
Refer to : https://huggingface.co/HuggingFaceTB/SmolLM2-1.7B-Instruct/blob/main/instructions_function_calling.md

In [1]:
import json
import re
from typing import Optional

from jinja2 import Template
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.utils import get_json_schema
from datetime import datetime
import random

##### Basic Tools

In [22]:
def get_current_time() -> str:
    """Returns the current time in 24-hour format.
    This function uses the datetime module to get the current time and formats it
    in HH:MM:SS format. The function does not take any arguments and returns a string
    representing the current time.
    The function uses the strftime() method to format the time.
    Args:
        None

    Returns:
        str: Current time in HH:MM:SS format.
    """
    return datetime.now().strftime("%H:%M:%S")


def get_random_number_between(min: int, max: int) -> int:
    """
    Gets a random number between min and max.
    This function uses the random.randint() method to generate a random integer
    between the specified minimum and maximum values (inclusive).
    The function takes two arguments: min and max, which define the range of the random number.
    
    Args:
        min: The minimum number.
        max: The maximum number.

    Returns:
        A random number between min and max.
    """
    return random.randint(min, max)


def get_number_to_ascii(number: int) -> str:
    """
    Converts a number to its ASCII representation using chr().
    This function takes an integer and returns the corresponding ASCII character.
    The function uses the built-in chr() function to convert the number to its ASCII character.

    Args:
        number: The number to convert.

    Returns:
        The ASCII representation of the number.
    """
    return chr(number)

##### System prompt using jinja2 template #####

In [3]:
basic_prompt = """
You are an expert in composing functions. You are given a question and a set of possible functions. 
Based on the question, you will need to make one or more function/tool calls to achieve the purpose. 
If none of the functions can be used, point it out and refuse to answer. 
If the given question lacks the parameters required by the function, also point it out.

You have access to the following tools:
<tools>{{ tools }}</tools>

The output MUST strictly adhere to the following format, and NO other text MUST be included.
The example format is as follows. Please make sure the parameter type is correct. If no function call is needed, please make the tool calls an empty list '[]'.
<tool_call>[
{"name": "func_name1", "arguments": {"argument1": "value1", "argument2": "value2"}},
{"name": "func_name2", "arguments": {"argument1": "value1", "argument2": "value2"}}

...... (multiple other function calls can be included here)
]</tool_call>
"""

system_prompt = Template(basic_prompt)

##### Next a function which takes  user query and list of tools and return the messages in chat format and also and model output parsing function


In [4]:
def prepare_messages(
    query: str,
    tools: Optional[dict[str, any]] = None,
    history: Optional[list[dict[str, str]]] = None,
) -> list[dict[str, str]]:
    """Prepare the system and user messages for the given query and tools.

    Args:
        query: The query to be answered.
        tools: The tools available to the user. Defaults to None, in which case if a
            list without content will be passed to the model.
        history: Exchange of messages, including the system_prompt from
            the first query. Defaults to None, the first message in a conversation.
    """
    if tools is None:
        tools = []
    if history:
        messages = history.copy()
        messages.append(
            {"role": "user", "content": query}
        )  # append the new query to the history which is a list of dicts
    else:
        messages = [
            {
                "role": "system",
                "content": system_prompt.render(tools=json.dumps(tools)),
            },
            {"role": "user", "content": query},
        ]
    return messages

In [5]:
def parse_response(text: str) -> str | dict[str, any]:
    """Parses a response from the model, returning either the
    parsed list with the tool calls parsed, or the
    model thought or response if couldn't generate one.

    Args:
        text: Response from the model.
    """
    pattern = r"<tool_call>(.*?)</tool_call>"
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        return json.loads(matches[0])
    return text

##### Setting the model and tokenizer

In [6]:
model_name_smollm = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name_smollm, device_map="auto", torch_dtype="auto", trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name_smollm)

/anaconda/envs/unstructured_seed_env/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


##### Next steps:
1. Prepare the input message to the model including the query and the tools
2. Pass the message conversation list to the model
3. Get the model response and parse it to get the function calls
4. Call the functions with the arguments provided by the model

In [7]:
tool_list = [
    get_json_schema(get_random_number_between),
    get_json_schema(get_current_time),
    get_json_schema(get_number_to_ascii),
]

In [8]:
# del tools
# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [9]:
tool_map = {
    "get_random_number_between": get_random_number_between,
    "get_current_time": get_current_time,
    "get_number_to_ascii": get_number_to_ascii,
}

In [10]:
query = "Get a random number between 1 and 10"

In [11]:
messages = prepare_messages(query, tool_list)
messages

[{'role': 'system',
  'content': '\nYou are an expert in composing functions. You are given a question and a set of possible functions. \nBased on the question, you will need to make one or more function/tool calls to achieve the purpose. \nIf none of the functions can be used, point it out and refuse to answer. \nIf the given question lacks the parameters required by the function, also point it out.\n\nYou have access to the following tools:\n<tools>[{"type": "function", "function": {"name": "get_random_number_between", "description": "Gets a random number between min and max.", "parameters": {"type": "object", "properties": {"min": {"type": "integer", "description": "The minimum number."}, "max": {"type": "integer", "description": "The maximum number."}}, "required": ["min", "max"]}, "return": {"type": "integer", "description": "A random number between min and max."}}}, {"type": "function", "function": {"name": "get_current_time", "description": "Returns the current time in 24-hour f

In [12]:
# pass the messages to the model to get the tool calls
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
result = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [13]:
result

'<tool_call>[{"name": "get_random_number_between", "arguments": {"min": 1, "max": 10}}]</tool_call>'

In [14]:
# Parse the response to extract tool calls
tool_calls = parse_response(result)
tool_calls

[{'name': 'get_random_number_between', 'arguments': {'min': 1, 'max': 10}}]

In [15]:
tool_responses = [
    tool_map.get(tc["name"])(*tc["arguments"].values()) for tc in tool_calls
]
tool_responses

[9]

In [16]:
# For the second turn, rebuild the history of messages:
history = messages.copy()
# Add the "parsed response"
history.append({"role": "assistant", "content": result})
history.append({"role": "assistant", "content": str(tool_responses)})
# Add the new query
query = "Can you give me the hour?"
messages = prepare_messages(query, tool_list, history)

messages

[{'role': 'system',
  'content': '\nYou are an expert in composing functions. You are given a question and a set of possible functions. \nBased on the question, you will need to make one or more function/tool calls to achieve the purpose. \nIf none of the functions can be used, point it out and refuse to answer. \nIf the given question lacks the parameters required by the function, also point it out.\n\nYou have access to the following tools:\n<tools>[{"type": "function", "function": {"name": "get_random_number_between", "description": "Gets a random number between min and max.", "parameters": {"type": "object", "properties": {"min": {"type": "integer", "description": "The minimum number."}, "max": {"type": "integer", "description": "The maximum number."}}, "required": ["min", "max"]}, "return": {"type": "integer", "description": "A random number between min and max."}}}, {"type": "function", "function": {"name": "get_current_time", "description": "Returns the current time in 24-hour f

In [18]:
# passing the messages to the model to get the tool calls
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
result = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
# Parse the response to extract tool calls
tool_calls = parse_response(result)
print(tool_calls)
# Call the tools with the parsed arguments
tool_responses = [tool_map.get(tc["name"])(*tc["arguments"].values()) for tc in tool_calls]
print(tool_responses)

[{'name': 'get_current_time', 'arguments': {}}]
['02:10:35']


In [19]:
# add tool call and response to the history
history.append({"role": "assistant", "content": result})
history.append({"role": "assistant", "content": str(tool_responses)})

#### Parallel function calls

In [21]:
query = "Can you give me the current hour and a random number between 1 and 50?"

messages = prepare_messages(query, tools=tool_list, history=history)
print(messages)

inputs = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
).to(model.device)
outputs = model.generate(
    inputs,
    max_new_tokens=512,
    do_sample=False,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
result = tokenizer.decode(outputs[0][len(inputs[0]) :], skip_special_tokens=True)

tool_calls = parse_response(result)
print(tool_calls)

[{'role': 'system', 'content': '\nYou are an expert in composing functions. You are given a question and a set of possible functions. \nBased on the question, you will need to make one or more function/tool calls to achieve the purpose. \nIf none of the functions can be used, point it out and refuse to answer. \nIf the given question lacks the parameters required by the function, also point it out.\n\nYou have access to the following tools:\n<tools>[{"type": "function", "function": {"name": "get_random_number_between", "description": "Gets a random number between min and max.", "parameters": {"type": "object", "properties": {"min": {"type": "integer", "description": "The minimum number."}, "max": {"type": "integer", "description": "The maximum number."}}, "required": ["min", "max"]}, "return": {"type": "integer", "description": "A random number between min and max."}}}, {"type": "function", "function": {"name": "get_current_time", "description": "Returns the current time in 24-hour for

[{'name': 'get_current_time', 'arguments': {}}, {'name': 'get_random_number_between', 'arguments': {'min': 1, 'max': 50}}]


In [23]:
# Call the tools with the parsed arguments
tool_responses = [
    tool_map.get(tc["name"])(*tc["arguments"].values()) for tc in tool_calls
]

In [24]:
tool_responses

['02:13:27', 32]

In [25]:
# Tools not available
query = "Can you open a new page with youtube?"

messages = prepare_messages(query, tools=tool_list, history=history)

inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
result = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

tool_calls = parse_response(result)

In [ ]:
tool_calls